In [1]:
%matplotlib inline 
from datetime import datetime 
from random import randrange 
import random
import operator 
import pandas as pd 
import matplotlib.pyplot as plt 
import re 
import numpy as np

In [ ]:
df = pd.read_csv('./data/capture20110811.pcap.netflow.labeled', sep ='\s+')
df.head()

/home/ilele/.conda/envs/cda/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [3]:
df[['Src_IP_Addr', 'Src_Port']] = df['Src_IP_Addr:Port'].str.split(':', n=1, expand=True)
df[['Dst_IP_Addr', 'Dst_Port']] = df['Dst_IP_Addr:Port'].str.split(':', n=1, expand=True)
#df['Date']= df['#Date_flow'].map(str)+ '' + df['start']
#df['Date']= pd.to_datetime(df['Date'], format = '%Y-%m-%d %H:%M:%S.%f', exact=True)
df['Label'] = df['Label(LEGITIMATE:Botnet:Background)']

df = df.drop(labels=['->', 'Label(LEGITIMATE:Botnet:Background)', 'Src_IP_Addr:Port', 'Dst_IP_Addr:Port'], axis=1)

print(df.shape)
print(df.columns)
df.head()

(6351188, 14)
Index(['#Date_flow', 'start', 'Durat', 'Prot', 'Flags', 'Tos', 'Packets',
       'Bytes', 'Flows', 'Src_IP_Addr', 'Src_Port', 'Dst_IP_Addr', 'Dst_Port',
       'Label'],
      dtype='object')


,#Date_flow,start,Durat,Prot,Flags,Tos,Packets,Bytes,Flows,Src_IP_Addr,Src_Port,Dst_IP_Addr,Dst_Port,Label
0,2011-08-11,10:10:00.003,0.967,UDP,INT,0,2,135,1,89.31.8.11,23929,147.32.84.229,13363,Background
1,2011-08-11,10:10:00.003,0.967,UDP,INT,0,2,276,1,147.32.84.229,13363,89.31.8.11,23929,Background
2,2011-08-11,10:10:00.006,0.000,UDP,INT,0,1,62,1,208.88.186.6,34042,147.32.84.229,13363,Background
3,2011-08-11,10:10:00.008,0.000,UDP,INT,0,1,78,1,92.118.218.77,55246,147.32.84.229,13363,Background
4,2011-08-11,10:10:00.009,0.000,UDP,INT,0,1,72,1,182.185.139.181,10223,147.32.84.229,13363,Background


In [4]:
print(df['Src_IP_Addr'].value_counts()[0:10])
infected_IP = '147.32.84.165'
val = df['Src_IP_Addr'].value_counts()[infected_IP] + df['Dst_IP_Addr'].value_counts()[infected_IP]
print("")
print("Infected ip occurrences: ", val)
# top 10 freq


147.32.84.229    1458791
147.32.80.9       573079
147.32.84.59      347413
147.32.84.138     331209
147.32.86.165      82589
147.32.84.165      45261
147.32.84.171      34186
147.32.84.118      34027
147.32.85.25       31279
147.32.86.20       28073
Name: Src_IP_Addr, dtype: int64

Infected ip occurrences:  54433


In [5]:
inf = df.loc[(df['Src_IP_Addr']== infected_IP) | (df['Dst_IP_Addr']== infected_IP)]
print(inf.shape)
inf.head()

(54433, 14)


,#Date_flow,start,Durat,Prot,Flags,Tos,Packets,Bytes,Flows,Src_IP_Addr,Src_Port,Dst_IP_Addr,Dst_Port,Label
540283,2011-08-11,10:27:20.087,0.000,UDP,INT,0,1,64,1,147.32.84.165,1025,147.32.80.9,53,Botnet
540285,2011-08-11,10:27:20.087,0.000,UDP,INT,0,1,139,1,147.32.80.9,53,147.32.84.165,1025,Botnet
541362,2011-08-11,10:27:22.334,0.000,UDP,INT,0,1,87,1,147.32.84.165,1025,147.32.80.9,53,Botnet
541376,2011-08-11,10:27:22.354,0.000,UDP,INT,0,1,503,1,147.32.80.9,53,147.32.84.165,1025,Botnet
541377,2011-08-11,10:27:22.355,0.045,TCP,SRPA_,0,4,629,1,147.32.84.165,1027,74.125.232.198,80,Botnet


In [9]:
def common_interactions(data,infected):
    
    ips={}
    for index, row in data.iterrows():
            if row["Src_IP_Addr"]==infected:
                if row["Dst_IP_Addr"] in ips:
                    ips[row["Dst_IP_Addr"]]+=1
                else:
                    ips[row["Dst_IP_Addr"]]=1
            else:
                if row["Src_IP_Addr"] in ips:
                    ips[row["Src_IP_Addr"]]+=1
                else:
                    ips[row["Src_IP_Addr"]]=1   
    ips = sorted(ips.items(), key=operator.itemgetter(1), reverse= True)
    top = ips[0:10]
    
    return top, ips   


In [10]:
top_10 , ips = common_interactions(inf, infected_IP)

print (np.array(top_10))


[['193.23.181.44' '6442']
 ['174.128.246.102' '4101']
 ['174.37.196.55' '3707']
 ['173.236.31.226' '3410']
 ['184.154.89.154' '3344']
 ['67.19.72.206' '3224']
 ['46.4.36.120' '3150']
 ['72.20.15.61' '3111']
 ['147.32.80.9' '1573']
 ['212.117.171.138' '967']]


In [13]:
inf['minmax']=0
inf["minmax"]=inf["minmax"].apply(lambda x:x+random.uniform(0,1))
inf.head()

C:\Users\konch\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\konch\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,#Date_flow,start,Durat,Prot,Flags,Tos,Packets,Bytes,Flows,Src_IP_Addr,Src_Port,Dst_IP_Addr,Dst_Port,Label,minmax
540283,2011-08-11,10:27:20.087,0.000,UDP,INT,0,1,64,1,147.32.84.165,1025,147.32.80.9,53,Botnet,0.039721
540285,2011-08-11,10:27:20.087,0.000,UDP,INT,0,1,139,1,147.32.80.9,53,147.32.84.165,1025,Botnet,0.243045
541362,2011-08-11,10:27:22.334,0.000,UDP,INT,0,1,87,1,147.32.84.165,1025,147.32.80.9,53,Botnet,0.044293
541376,2011-08-11,10:27:22.354,0.000,UDP,INT,0,1,503,1,147.32.80.9,53,147.32.84.165,1025,Botnet,0.197324
541377,2011-08-11,10:27:22.355,0.045,TCP,SRPA_,0,4,629,1,147.32.84.165,1027,74.125.232.198,80,Botnet,0.549755


In [15]:
k=[10,50,100,500,1000,5000,10000]

Inf=inf.sort_values("minmax",ascending=False)
for k in k:
    print(" For k:", k)
    sample= inf.nlargest(k,'minmax')
    ten, ip = common_interactions(sample,infected_IP)
    print(np.array(ten))
    print(" ")

 For k: 10
[['173.236.31.226' '2']
 ['205.188.156.193' '1']
 ['184.154.89.154' '1']
 ['64.12.90.66' '1']
 ['174.128.246.102' '1']
 ['207.115.20.23' '1']
 ['46.4.36.120' '1']
 ['212.117.171.138' '1']
 ['65.54.188.110' '1']]
 
 For k: 50
[['193.23.181.44' '6']
 ['72.20.15.61' '5']
 ['46.4.36.120' '4']
 ['173.236.31.226' '3']
 ['184.154.89.154' '2']
 ['174.128.246.102' '2']
 ['147.32.80.9' '2']
 ['67.19.72.206' '2']
 ['64.12.90.97' '2']
 ['205.188.156.193' '1']]
 
 For k: 100
[['193.23.181.44' '9']
 ['174.128.246.102' '8']
 ['72.20.15.61' '7']
 ['46.4.36.120' '6']
 ['174.37.196.55' '5']
 ['184.154.89.154' '4']
 ['147.32.80.9' '4']
 ['67.19.72.206' '4']
 ['173.192.170.88' '4']
 ['173.236.31.226' '3']]
 
 For k: 500
[['193.23.181.44' '65']
 ['174.128.246.102' '36']
 ['67.19.72.206' '32']
 ['72.20.15.61' '31']
 ['174.37.196.55' '29']
 ['173.236.31.226' '28']
 ['46.4.36.120' '26']
 ['184.154.89.154' '25']
 ['147.32.80.9' '11']
 ['173.192.170.88' '11']]
 
 For k: 1000
[['193.23.181.44' '122']


In [48]:
import hashlib
import mmh3

width = 10
depth = 5
#seeds = np.random.choice(100, depth, replace=False)
print(seeds)

cm = np.zeros((depth, width), dtype=int)

mmh3.hash(tt.values[0], seeds[0]) % width

[85  9 24 72 35]


1

In [49]:
for t in tt.values:
    for d in range(depth):
        hash_index = mmh3.hash(t, seeds[d], signed=False) % width
        cm[d, hash_index] = cm[d, hash_index] + 1
print(cm)

[[ 1699 47018  2546   785  1113   185   181   133   546   227]
 [  543    28 46013  1773  2639   769   450    66   325  1827]
 [  333  1639   160  2121   915 45688  1321   119   110  2027]
 [  190   587   270  1623    88  2203   746   106  2039 46581]
 [  541   866   720 45391  2597    86    93   613  3451    75]]


In [53]:
key = tt.values[100]
print(key)

estim = np.ones(depth, dtype=int) * np.max(cm)
for d in range(depth):
    hash_index = mmh3.hash(t, seeds[d], signed=False) % width
    if cm[d, hash_index] < estim[d]:
        estim[d] = cm[d,hash_index]
        
print(estim)
print(min(estim))

147.32.80.9
[ 546  543 1321  587  866]
543
